In [8]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [9]:
cwl_dir = '../cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [12]:
template = {
    'fq_1': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'reference': {
        'class': 'File',
        'path': 'path/to/GRCh38.d1.vd1.fa',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.amb'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.ann'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.bwt'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.fai'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.pac'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.fa.sa'},
            {'class': 'File', 'path': 'path/to/GRCh38.d1.vd1.dict'}
        ]
    },
    'known_sites': {
        'class': 'File',
        'path': 'path/to/vcf.gz',
        'secondaryFiles': [
            {'class': 'File', 'path': 'path/to/vcf.gz.tbi'}
        ]
    },
    'sample': 'a_string',
    'flowcell': 'a_string',
    'lane': 'a_string',
    'index_sequencer': 'a_string',
    'library_preparation': 'a_string',
    'platform': 'a_string',
    'sample': 'a_string',
    'cpu': 'a_int'
}

In [13]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.align_dnaseq.yaml'), 'w'))

In [15]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    'id': 'align_dnaseq',
    'label': 'align_dnaseq',
    'baseCommand': ['python', '/align_dnaseq/align_dnaseq/align_dnaseq.py'],
    'inputs': [
        {
            'id': 'sample',
            'type': 'string',
            'inputBinding': {
                'position': '1'
            }
        },
        {
            'id': 'fq1',
            'type': 'File',
            'inputBinding': {
                'position': '2'
            }
        },
        {
            'id': 'fq2',
            'type': 'File',
            'inputBinding': {
                'position': '3'
            }
        },
        {
            'id': 'flowcell',
            'type': 'string',
            'inputBinding': {
                'prefix': '--flowcell',
                'position': '0'
            }
        },
        {
            'id': 'lane',
            'type': 'string',
            'inputBinding': {
                'prefix': '--lane',
                'position': '0'
            }
        },
         {
            'id': 'index_sequencer',
            'type': 'string',
            'inputBinding': {
                'prefix': '--index-sequencer',
                'position': '0'
            }
        },
         {
            'id': 'library_preparation',
            'type': 'string',
            'inputBinding': {
                'prefix': '--library-preparation',
                'position': '0'
            }
        },
        {
            'id': 'platform',
            'type': 'string',
            'inputBinding': {
                'prefix': '--platform',
                'position': '0'
            }
        },
        {
            'id': 'known-sites',
            'type': 'File',
            'secondaryFiles': ['.tbi'],
            'inputBinding': {
                'prefix': '--known-sites',
                'position': '0'
            }
         },
         {
            'id': 'reference',
            'type': 'File',
            'secondaryFiles': [
                '.amb',
                '.ann',
                '.bwt',
                '.fai',
                '.pac',
                '.sa',
                '^.dict'
            ],
            'inputBinding': {
                'prefix': '--reference',
                'position': '0'
            }
        },
        # needs path to be set so it works on compute1
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/align_dnaseq/bin:$PATH'
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--out-prefix',
            'valueFrom': 'output'
        },
    ],
    'outputs': [
        {
            'id': 'output_bam',
            'type': 'File',
            'outputBinding': {'glob': 'output.bam'}
        }
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/align_dnaseq:0.0.1'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 28000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
            }
        }
    ]
}

In [16]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'align_dnaseq.cwl'), 'w'))